# Text-to-Image Generation with a Pretrained Diffusion Model

Этот ноутбук показывает, как собрать простой пайплайн текст-в-изображение с использованием предобученной диффузионной модели. Структура следует типичному рабочему процессу: сначала объясняем основные блоки, затем демонстрируем отдельные шаги (токенизация текста, работа шумового расписания), и завершаем генерацией изображения по заданному описанию.

## 1. Подготовка окружения и зависимостей

В диффузионных моделях нам нужны библиотеки `diffusers`, `transformers` и `torch`. Этот блок ставит зависимости и настраивает устройство выполнения (CPU/GPU).

In [ ]:
# Установка зависимостей (раскомментируйте при первом запуске)
# !pip install -q diffusers transformers accelerate torch torchvision matplotlib


## 2. Импорты и выбор вычислительного устройства

Здесь подключаем необходимые модули, а также определяем, есть ли доступная GPU для ускорения инференса.

In [ ]:
import torch
from diffusers import StableDiffusionPipeline
import matplotlib.pyplot as plt

# Определяем устройство: если есть CUDA, используем её для ускорения
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
DTYPE = torch.float16 if DEVICE == 'cuda' else torch.float32

print(f'Using device: {DEVICE}, dtype: {DTYPE}')


## 3. Загрузка предобученной модели

Мы используем компактную модель [`stabilityai/sdxl-turbo`](https://huggingface.co/stabilityai/sdxl-turbo), оптимизированную для быстрого инференса. Здесь создаём пайплайн и переносим его на выбранное устройство.

In [ ]:
model_id = 'stabilityai/sdxl-turbo'
variant = 'fp16' if DTYPE == torch.float16 else None

pipe = StableDiffusionPipeline.from_pretrained(
    model_id,
    torch_dtype=DTYPE,
    variant=variant,
)
pipe = pipe.to(DEVICE)

# Для воспроизводимости задаём сид
generator = torch.Generator(device=DEVICE).manual_seed(42)
print('Pipeline is ready')


## 4. Блок токенизации текста

Диффузионная модель принимает векторное представление текста. Этот блок показывает, как токенизатор превращает строку в тензоры, пригодные для подачи в текстовый энкодер.

In [ ]:
def tokenize_prompt(prompt: str):
    '''Возвращает токенизированный запрос и сопутствующие тензоры.'''
    tokenized = pipe.tokenizer(
        prompt,
        padding='max_length',
        truncation=True,
        max_length=pipe.tokenizer.model_max_length,
        return_tensors='pt',
    )
    return tokenized

sample_prompt = 'a serene watercolor landscape of mountains at sunrise'
encoded = tokenize_prompt(sample_prompt)
print('Input IDs shape:', encoded.input_ids.shape)
print('Attention mask shape:', encoded.attention_mask.shape)


## 5. Шумовое расписание и денойзинг (обзор)

Диффузионные модели постепенно удаляют шум из латентного представления. В этом блоке мы смотрим на расписание шагов и иллюстрируем добавление шума на одном из таймстепов. Это помогает понять, как работает прямой процесс (добавление шума), тогда как обратный процесс (денойзинг) выполняется моделью автоматически.

In [ ]:
# Получаем список таймстепов из шедулера
scheduler = pipe.scheduler
print('Number of timesteps:', len(scheduler.timesteps))

# Демонстрация добавления шума к случайным латентам
latent_shape = (1, pipe.unet.config.in_channels, pipe.unet.config.sample_size, pipe.unet.config.sample_size)
base_latents = torch.randn(latent_shape)
noise = torch.randn_like(base_latents)
step_index = len(scheduler.timesteps) // 2
noisy_latents = scheduler.add_noise(base_latents, noise, scheduler.timesteps[step_index])

# Для наглядности покажем распределение значений до и после добавления шума
fig, axes = plt.subplots(1, 2, figsize=(10, 4))
axes[0].hist(base_latents.flatten().numpy(), bins=40, color='steelblue')
axes[0].set_title('Базовые латенты')
axes[1].hist(noisy_latents.flatten().numpy(), bins=40, color='darkorange')
axes[1].set_title('Зашумлённые латенты')
plt.tight_layout()
plt.show()


## 6. Генерация изображения по текстовому описанию

Теперь запускаем полный пайплайн: передаём запрос, выполняем несколько итераций денойзинга и визуализируем результат. Модель `sdxl-turbo` хорошо работает с 4 шагами и нулевой классификационной свободой (`guidance_scale=0`).

In [ ]:
final_prompt = 'a cozy cottage beside a lake under a starry night sky, watercolor illustration'

image = pipe(
    final_prompt,
    num_inference_steps=4,
    guidance_scale=0.0,
    generator=generator,
).images[0]

# Сохраняем и показываем результат
output_path = 'generated_sample.png'
image.save(output_path)
print(f'Saved image to {output_path}')

plt.figure(figsize=(6, 6))
plt.imshow(image)
plt.axis('off')
plt.title(final_prompt)
plt.show()


## 7. Итоги

Мы прошли через ключевые этапы текст-ту-изображение диффузионного пайплайна: разобрались, зачем нужен каждый блок, посмотрели на токенизацию и шумовое расписание, а затем сгенерировали финальное изображение с помощью предобученной модели. Чтобы адаптировать ноутбук под свои задачи, измените текстовый запрос или выберите другую модель в `model_id`.